<a href="https://colab.research.google.com/github/zephyris/discoba_alphafold/blob/main/CommunityPredictionsBatch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Batch AlphaFold predictions for TriTrypDB

#@markdown This Colab notebook carries out protein structure predictions for proteins using just a gene ID from TriTrypDB.
#@markdown It runs in a batch mode, automatically splitting up large proteins to manageable chunks and can process multiple proteins.

#@markdown All predictions are  shared to a community database - this avoids re-predicting proteins.
#@markdown It also means your predictions aren't lost if your session times out.
#@markdown If you want to keep your predictions private you'll have to do the prediction manually with [one of my other Colab notebooks](https://github.com/zephyris/discoba_alphafold).

#@markdown **Enter your TriTrypDB gene ID here.**
#@markdown You can also paste a list of gene IDs, separated by spaces or commas.

gene_ids = '' #@param {type:"string"}
gene_ids = gene_ids.replace(",", " ")
gene_ids = '\n'.join(gene_ids.split())+"\n"
f = open("queries.txt", "w")
f.write(gene_ids)
f.close()

#@markdown Press <kbd>Ctrl</kbd> + <kbd>F9</kbd> or select `Runtime` > `Run All` to carry out predictions.
#@markdown You should also make sure a GPU is selected under `Runtime` > `Change runtime type` too!

#@markdown It'll keep predicting until Google decides you've had enough computing time... they do donate it for free after all.
#@markdown The more you interact with the page the longer that will be, so looking at progress genuinely helps, especially for big proteins.

#@markdown If you get the `Runtime disconnected` message just press `Reconnect` then hit <kbd>Ctrl</kbd> + <kbd>F9</kbd> and run it again.
#@markdown Unfortunately, if you only get the option to `Connect without GPU`, you'll have to wait - normally a day or so.

#@markdown If you want to avoid timeouts you can get a [Colab Pro subscription](https://colab.research.google.com/signup), but that costs a monthly fee.

from google.colab import files
import os
import os.path
import re
import hashlib
import urllib

def add_hash(x,y):
  return x+"_"+hashlib.sha1(y.encode()).hexdigest()[:5]

#TriTrypDB_Gene_ID = 'Tb927.1.380' #@param {type:"string"}
#jobname = TriTrypDB_Gene_ID
#print("Predicting "+TriTrypDB_Gene_ID)

#Submit_to_community_resource = True #@param {type:"boolean"}

#@markdown _Optional:_ If you'd like to have your efforts recorded put a username here. Feel free to work as a team and share a user name, just make sure you pick something unique! This'll get recorded in the prediction file as a thank you.
user_name = '' #@param {type:"string"}

# settings
num_models = 5
use_msa = True
use_env = True
use_amber = False
use_templates = False
homooligomer = 1
max_length = 800

input_dir = 'queries'
result_dir = 'results'
do_not_overwite_results = True

print("Predicting these genes IDs:")
print(gene_ids.replace(" ", "\n"))
print("Thank you for sharing your results with the community!")
if user_name!="":
  print(f"'{user_name}' will be listed to say thanks :)")

In [ ]:
#@title Get everything installed

#@markdown This doesn't actually install anything on your computer, just on Google's server.

%%bash -s $use_amber $use_msa $use_templates $jobname $use_env

USE_AMBER=$1
USE_MSA=$2
USE_TEMPLATES=$3
JOBNAME=$4
USE_ENV=$5

# From ColabFold batch
if [ ! -f AF2_READY ]; then
  # install dependencies
  pip -q install biopython dm-haiku ml-collections py3Dmol
  wget -qnc https://raw.githubusercontent.com/sokrypton/ColabFold/main/beta/colabfold.py
  echo "Installed ColabFold dependencies"

  # download model
  if [ ! -d "alphafold/" ]; then
    git clone https://github.com/deepmind/alphafold.git --quiet
    (cd alphafold; git checkout 0bab1bf84d9d887aba5cfb6d09af1e8c3ecbc408 --quiet)
    mv alphafold alphafold_
    mv alphafold_/alphafold .
    # remove "END" from PDBs, otherwise biopython complains
    sed -i "s/pdb_lines.append('END')//" /content/alphafold/common/protein.py
    sed -i "s/pdb_lines.append('ENDMDL')//" /content/alphafold/common/protein.py
    echo "Downloaded AlphaFold model"
  fi

  # download model params (~1 min)
  if [ ! -d "params/" ]; then
    mkdir params
    curl -fsSL https://storage.googleapis.com/alphafold/alphafold_params_2021-07-14.tar \
    | tar x -C params
  fi
  touch AF2_READY
  echo "Downloaded AlphaFold model parameters"
fi
# download libraries for interfacing with MMseqs2 API
if [ ${USE_MSA} == "True" ] || [ ${USE_TEMPLATES} == "True" ]; then
  if [ ! -f MMSEQ2_READY ]; then
    apt-get -qq -y update 2>&1 1>/dev/null
    apt-get -qq -y install jq curl zlib1g gawk 2>&1 1>/dev/null
    touch MMSEQ2_READY
    echo "Downloaded MMSeqs2 libraries"
  fi
fi
# setup conda
if [ ${USE_AMBER} == "True" ] || [ ${USE_TEMPLATES} == "True" ]; then
  if [ ! -f CONDA_READY ]; then
    wget -qnc https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
    bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local 2>&1 1>/dev/null
    rm Miniconda3-latest-Linux-x86_64.sh
    touch CONDA_READY
    echo "Downloaded Amber and dependencies"
  fi
fi
# setup template search
if [ ${USE_TEMPLATES} == "True" ] && [ ! -f HH_READY ]; then
  conda install -y -q -c conda-forge -c bioconda kalign3=3.2.2 hhsuite=3.3.0 python=3.7 2>&1 1>/dev/null
  touch HH_READY
  echo "Setup template search"
fi
# setup openmm for amber refinement
if [ ${USE_AMBER} == "True" ] && [ ! -f AMBER_READY ]; then
  conda install -y -q -c conda-forge openmm=7.5.1 python=3.7 pdbfixer 2>&1 1>/dev/null
  (cd /usr/local/lib/python3.7/site-packages; patch -s -p0 < /content/alphafold_/docker/openmm.patch)
  wget -qnc https://git.scicore.unibas.ch/schwede/openstructure/-/raw/7102c63615b64735c4941278d92b554ec94415f8/modules/mol/alg/src/stereo_chemical_props.txt
  mv stereo_chemical_props.txt alphafold/common/
  touch AMBER_READY
  echo "Downloaded OpenMM for Amber"
fi

# Discoba-specific
# install HMMER
if [ ! -f HMMER_READY ]; then
  apt-get install hmmer > /dev/null 2>&1
  touch HMMER_READY
  echo "Installed HMMER"
fi
# download the custom Discoba database
if [ ! -f DISCOBA_READY ]; then
  if [ -d discoba ]; then
    rm -r discoba
  fi
  mkdir discoba
  cd discoba
    curl http://wheelerlab.net/discobaStats.txt -s -o discobaStats.txt
    curl http://wheelerlab.net/discoba.fasta.gz -s -L -o discoba.fasta.gz
    gzip -d discoba.fasta.gz
  cd ..
  touch DISCOBA_READY
  echo "Downloaded Discoba database"
fi
# install HH-suite
if [ ! -f HHSUITE_READY ]; then
  if [ -d hh-suite ]; then
    rm -r hh-suite
  fi
  git clone https://github.com/soedinglab/hh-suite --quiet
  touch HHSUITE_READY
  echo "Installed HH-suite"
fi
# install IUPred
if [ ! -f IUPRED_READY ]; then
  curl https://raw.githubusercontent.com/zephyris/discoba_alphafold/main/scripts/iupredCut.py -s -o iupredCut.py
  curl https://iupred.elte.hu/static/bin/iupred3.tar.gz -s -o iupred3.tar.gz
  tar -xf iupred3.tar.gz
  # pip install scipy
  touch IUPRED_READY
  echo "Installed IUPred"
fi

echo ""
echo "All the software is ready to go... let's get predictin'."

In [ ]:
#@title Prep proteins for predictions

#@markdown This gets the protein sequences from TriTrypDB and preps them.

#@markdown If structures are already available you'll see the links listed below.

%%bash -s $max_length

MAXLENGTH=$1

# setup worklist
if [ ! -d queries ]; then
  mkdir queries
else
  rm queries/*
fi

# fetch sequences
while read P; do
  curl "https://tritrypdb.org/tritrypdb/service/record-types/gene/searches/single_record_question_GeneRecordClasses_GeneRecordClass/reports/srt" -s \
    --data-raw "data=%7B%22searchConfig%22%3A%7B%22parameters%22%3A%7B%22primaryKeys%22%3A%22"$P"%2CTriTrypDB%22%7D%7D%2C%22reportConfig%22%3A%7B%22attachmentType%22%3A%22plain%22%2C%22endOffset3%22%3A0%2C%22type%22%3A%22protein%22%2C%22sourceIdFilter%22%3A%22genesOnly%22%2C%22upstreamAnchor%22%3A%22Start%22%2C%22upstreamSign%22%3A%22plus%22%2C%22upstreamOffset%22%3A0%2C%22downstreamAnchor%22%3A%22End%22%2C%22downstreamSign%22%3A%22plus%22%2C%22downstreamOffset%22%3A0%2C%22onlyIdDefLine%22%3A%220%22%2C%22noLineBreaks%22%3A%220%22%2C%22startAnchor3%22%3A%22Start%22%2C%22startOffset3%22%3A0%2C%22endAnchor3%22%3A%22End%22%7D%7D" \
    > "queries/"$P".fasta"
done < queries.txt

# use IUPred to find disordered regions
for P in queries/*.fasta; do
  python iupred3/iupred3.py $P short | grep "^[^#;]" > $P".iupred"
done

# split FASTAs to jobs
echo "Assigned predictions for today:"
echo "Protein FASTA files"
for P in queries/*.fasta; do
  LENGTH=$(cat $P".iupred" | wc -l)
  echo $P "("$LENGTH" aa)"
  if [ $LENGTH -gt $MAXLENGTH ]; then
    rm $P
    python3 iupredCut.py $P".iupred" $MAXLENGTH
  else
    mv $P ${P%.fasta}"_1-"$LENGTH".fasta"
  fi
done

# remove IUPred results
rm queries/*.iupred

echo ""
echo "FASTA files split into workable chunks"
# summarise
for P in queries/*.fasta; do
  FILE=$(basename -- $P)
  STATUS=$(curl -I -s "http://www.wheelerlab.net/check.php?idse="${FILE%.fasta} | head -n1 | awk '{print $2}')
  LENGTH=$(cat $P | tail -n+2 | sed 's/^[[:space:]]*//g' | wc -c)
  if [ $STATUS = "404" ]; then
    echo $P "("$LENGTH" aa)"
  else
    rm $P
    echo $P" already predicted, removed from your workload!"
  fi
done

# remove FASTAs with identical sequences
echo ""
echo "Checking for duplicate sequences"
for P in queries/*.fasta; do
  for Q in queries/*.fasta; do
    if [ $P != $Q ] && [ -f $P ] && [ -f $Q ]; then
      PSEQ=$(cat $P | tail -n 1)
      QSEQ=$(cat $Q | tail -n 1)
      if [ $PSEQ == $QSEQ ]; then
        echo $Q" has the same sequence as "$P", removed from your workload!"
        rm $Q
      fi
    fi
  done
done

In [ ]:
#@title Predict some structures!

#@markdown Predictions are made one-by-one, starting with the shortest sequence.
#@markdown As soon as each prediction finishes the results are automatically uploaded to the database server.
#@markdown It sends absolutely no personal information, just the prediction result (and your user name, if you picked one).

#@markdown Because this runs in batch mode there can be many sets of results files. 
#@markdown It's not really possible to download the results automatically - auto-downloading lots of files looks like a virus!
#@markdown Instead, you can manually grab the results using the file browser on the interface (click on the little folder icon on the left).

#@markdown Or... just follow the links to the database website!

#@markdown Occasionally errors happen which stop the predictions (the little play button for this section goes red). Just hit <kbd>Ctrl</kbd> + <kbd>F9</kbd> to restart, and you can always select `Runtime` > `Factory reset runtime` to do a full reset of everything.

print ("You're now running AlphaFold!")
print ("")

# setup the model
if "model" not in dir():

  # hiding warning messages
  import warnings
  from absl import logging
  import os
  import tensorflow as tf
  warnings.filterwarnings('ignore')
  logging.set_verbosity("error")
  os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
  tf.get_logger().setLevel('ERROR')

  import sys
  import numpy as np
  import pickle
  from alphafold.common import protein
  from alphafold.data import pipeline
  from alphafold.data import templates
  from alphafold.model import data
  from alphafold.model import config
  from alphafold.model import model
  from alphafold.data.tools import hhsearch
  from alphafold.model.tf import shape_placeholders
  NUM_RES = shape_placeholders.NUM_RES
  NUM_MSA_SEQ = shape_placeholders.NUM_MSA_SEQ
  NUM_EXTRA_SEQ = shape_placeholders.NUM_EXTRA_SEQ
  NUM_TEMPLATES = shape_placeholders.NUM_TEMPLATES

  import colabfold as cf
  # plotting libraries
  import py3Dmol
  import matplotlib.pyplot as plt
  import ipywidgets
  from ipywidgets import interact, fixed, GridspecLayout, Output

  import json

if use_amber and "relax" not in dir():
  sys.path.insert(0, '/usr/local/lib/python3.7/site-packages/')
  from alphafold.relax import relax

def mk_mock_template(query_sequence, num_temp=1):
  ln = len(query_sequence)
  output_templates_sequence = "A"*ln
  output_confidence_scores = np.full(ln,1.0)
  templates_all_atom_positions = np.zeros((ln, templates.residue_constants.atom_type_num, 3))
  templates_all_atom_masks = np.zeros((ln, templates.residue_constants.atom_type_num))
  templates_aatype = templates.residue_constants.sequence_to_onehot(output_templates_sequence,
                                                                    templates.residue_constants.HHBLITS_AA_TO_ID)
  template_features = {'template_all_atom_positions': np.tile(templates_all_atom_positions[None],[num_temp,1,1,1]),
                       'template_all_atom_masks': np.tile(templates_all_atom_masks[None],[num_temp,1,1]),
                       'template_sequence': [f'none'.encode()]*num_temp,
                       'template_aatype': np.tile(np.array(templates_aatype)[None],[num_temp,1,1]),
                       'template_confidence_scores': np.tile(output_confidence_scores[None],[num_temp,1]),
                       'template_domain_names': [f'none'.encode()]*num_temp,
                       'template_release_date': [f'none'.encode()]*num_temp}
  return template_features


def mk_template(a3m_lines, template_paths):
  template_featurizer = templates.TemplateHitFeaturizer(
      mmcif_dir=template_paths,
      max_template_date="2100-01-01",
      max_hits=20,
      kalign_binary_path="kalign",
      release_dates_path=None,
      obsolete_pdbs_path=None)

  hhsearch_pdb70_runner = hhsearch.HHSearch(binary_path="hhsearch", databases=[f"{template_paths}/pdb70"])

  hhsearch_result = hhsearch_pdb70_runner.query(a3m_lines)
  hhsearch_hits = pipeline.parsers.parse_hhr(hhsearch_result)
  templates_result = template_featurizer.get_templates(query_sequence=query_sequence,
                                                       query_pdb_code=None,
                                                       query_release_date=None,
                                                       hits=hhsearch_hits)
  return templates_result.features

def make_fixed_size(protein, shape_schema, msa_cluster_size, extra_msa_size,
                   num_res, num_templates=0):
  """Guess at the MSA and sequence dimensions to make fixed size."""

  pad_size_map = {
      NUM_RES: num_res,
      NUM_MSA_SEQ: msa_cluster_size,
      NUM_EXTRA_SEQ: extra_msa_size,
      NUM_TEMPLATES: num_templates,
  }

  for k, v in protein.items():
    # Don't transfer this to the accelerator.
    if k == 'extra_cluster_assignment':
      continue
    shape = list(v.shape)
    
    schema = shape_schema[k]

    assert len(shape) == len(schema), (
        f'Rank mismatch between shape and shape schema for {k}: '
        f'{shape} vs {schema}')
    pad_size = [
        pad_size_map.get(s2, None) or s1 for (s1, s2) in zip(shape, schema)
    ]
    padding = [(0, p - tf.shape(v)[i]) for i, p in enumerate(pad_size)]

    if padding:
      protein[k] = tf.pad(
          v, padding, name=f'pad_to_fixed_{k}')
      protein[k].set_shape(pad_size)
  return {k:np.asarray(v) for k,v in protein.items()}

def set_bfactor(pdb_filename, bfac, idx_res, chains):
  I = open(pdb_filename,"r").readlines()
  O = open(pdb_filename,"w")
  for line in I:
    if line[0:6] == "ATOM  ":
      seq_id = int(line[22:26].strip()) - 1
      seq_id = np.where(idx_res == seq_id)[0][0]
      O.write(f"{line[:21]}{chains[seq_id]}{line[22:60]}{bfac[seq_id]:6.2f}{line[66:]}")
  O.close()

def predict_structure(prefix, feature_dict, Ls, crop_len, model_params, use_model, do_relax=False, random_seed=0):  
  """Predicts structure using AlphaFold for the given sequence."""
  idx_res = feature_dict['residue_index']
  chains = list("".join([ascii_uppercase[n]*L for n,L in enumerate(Ls)]))

  # Run the models.
  plddts,paes = [],[]
  unrelaxed_pdb_lines = []
  relaxed_pdb_lines = []
  seq_len = feature_dict['seq_length'][0]
  for model_name, params in model_params.items():
    if model_name in use_model:
      print(f"running {model_name}")
      # swap params to avoid recompiling
      # note: models 1,2 have diff number of params compared to models 3,4,5
      if any(str(m) in model_name for m in [1,2]): model_runner = model_runner_1
      if any(str(m) in model_name for m in [3,4,5]): model_runner = model_runner_3
      model_runner.params = params

      processed_feature_dict = model_runner.process_features(feature_dict, random_seed=random_seed)
      model_config = model_runner.config
      eval_cfg = model_config.data.eval
      crop_feats = {k:[None]+v for k,v in dict(eval_cfg.feat).items()}     

      # templates models
      if model_name == "model_1" or model_name == "model_2":
        pad_msa_clusters = eval_cfg.max_msa_clusters - eval_cfg.max_templates
      else:
        pad_msa_clusters = eval_cfg.max_msa_clusters

      max_msa_clusters = pad_msa_clusters

      # let's try pad (num_res + X) 
      input_fix = make_fixed_size(processed_feature_dict,
                                  crop_feats,
                                  msa_cluster_size=max_msa_clusters, # true_msa (4, 512, 68)
                                  extra_msa_size=5120, # extra_msa (4, 5120, 68)
                                  num_res=crop_len, # aatype (4, 68)
                                  num_templates=4) # template_mask (4, 4) second value

      prediction_result = model_runner.predict(input_fix)
      unrelaxed_protein = protein.from_prediction(input_fix,prediction_result)
      unrelaxed_pdb_lines.append(protein.to_pdb(unrelaxed_protein))
      plddts.append(prediction_result['plddt'][:seq_len])
      paes_res = []
      for i in range(seq_len):
        paes_res.append(prediction_result['predicted_aligned_error'][i][:seq_len])
      paes.append(paes_res)
      if do_relax:
        # Relax the prediction.
        amber_relaxer = relax.AmberRelaxation(max_iterations=0,tolerance=2.39,
                                              stiffness=10.0,exclude_residues=[],
                                              max_outer_iterations=20)      
        relaxed_pdb_str, _, _ = amber_relaxer.process(prot=unrelaxed_protein)
        relaxed_pdb_lines.append(relaxed_pdb_str)

  # rerank models based on predicted lddt
  lddt_rank = np.mean(plddts,-1).argsort()[::-1]
  out = {}
  print("reranking models based on avg. predicted lDDT")
  for n,r in enumerate(lddt_rank):
    print(f"model_{n+1} {np.mean(plddts[r])}")

    unrelaxed_pdb_path = f'{prefix}_unrelaxed_model_{n+1}.pdb'    
    with open(unrelaxed_pdb_path, 'w') as f: f.write(unrelaxed_pdb_lines[r])
    set_bfactor(unrelaxed_pdb_path, plddts[r], idx_res, chains)

    if do_relax:
      relaxed_pdb_path = f'{prefix}_relaxed_model_{n+1}.pdb'
      with open(relaxed_pdb_path, 'w') as f: f.write(relaxed_pdb_lines[r])
      set_bfactor(relaxed_pdb_path, plddts[r], idx_res, chains)

    out[f"model_{n+1}"] = {"plddt":plddts[r], "pae":paes[r], "unrelaxedPdb":unrelaxed_pdb_lines[r]}
    if do_relax:
      out[f"model_{n+1}"]["relaxedPdb"]=relaxed_pdb_lines[r]
  return out

from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(input_dir) if isfile(join(input_dir, f))]
queries = []
for filename in onlyfiles:
    extension = os.path.splitext(filename)[1]
    jobname=os.path.splitext(filename)[0]
    filepath = input_dir+"/"+filename
    with open(filepath) as f:
      input_fasta_str = f.read()
    (seqs, header) = pipeline.parsers.parse_fasta(input_fasta_str)
    query_sequence = seqs[0]
    queries.append((jobname, query_sequence, extension))
# sort by seq. len    
queries.sort(key=lambda t: len(t[1]))
import math
crop_len = math.ceil(len(queries[0][1]) * 1.1)

### run
for (jobname, query_sequence, extension) in queries:
  a3m_file = f"{jobname}.a3m"
  if len(query_sequence) > crop_len:
    crop_len = math.ceil(len(query_sequence) * 1.1)
  print("Running: "+jobname)
  if do_not_overwite_results == True and os.path.isfile(result_dir+"/"+jobname+".result.zip"):
    continue
  if use_templates:
    try:  
      a3m_lines, template_paths = cf.run_mmseqs2(query_sequence, jobname, use_env, use_templates=True)
    except:
      print(jobname+" cound not be processed")
      continue  
    if template_paths is None:
      template_features = mk_mock_template(query_sequence, 100)
    else:
      template_features = mk_template(a3m_lines, template_paths)
    if extension.lower() == ".a3m":
      a3m_lines = "".join(open(input_dir+"/"+a3m_file,"r").read())
  else:
    if extension.lower() == ".a3m":
      a3m_lines = "".join(open(input_dir+"/"+a3m_file,"r").read())
    else:
      try:  
        a3m_lines = cf.run_mmseqs2(query_sequence, jobname, use_env)
      except:
        print(jobname+" cound not be processed")
        continue
    template_features = mk_mock_template(query_sequence, 100)

  with open(a3m_file, "w") as text_file:
    text_file.write(a3m_lines)

  #Add discoba
  %shell head -n2 $a3m_file > tmp.fasta
  %shell jackhmmer -A tmp.sto -o tmp.out tmp.fasta discoba/discoba.fasta
  %shell perl hh-suite/scripts/reformat.pl sto a3m tmp.sto tmp.a3m -l 30000 2>&1 1>/dev/null
  %shell cat tmp.a3m | tail -n+1 >> $a3m_file
  with open(a3m_file, "r") as text_file:
    a3m_lines=text_file.read()

  # parse MSA
  msa, deletion_matrix = pipeline.parsers.parse_a3m(a3m_lines)

  #@title Gather input features, predict structure
  from string import ascii_uppercase

  # collect model weights
  use_model = {}
  if "model_params" not in dir(): model_params = {}
  for model_name in ["model_1","model_2","model_3","model_4","model_5"][:num_models]:
    use_model[model_name] = True
    if model_name not in model_params:
      model_params[model_name] = data.get_model_haiku_params(model_name=model_name+"_ptm", data_dir=".")
      if model_name == "model_1":
        model_config = config.model_config(model_name+"_ptm")
        model_config.data.eval.num_ensemble = 1
        model_runner_1 = model.RunModel(model_config, model_params[model_name])
      if model_name == "model_3":
        model_config = config.model_config(model_name+"_ptm")
        model_config.data.eval.num_ensemble = 1
        model_runner_3 = model.RunModel(model_config, model_params[model_name])


  msas = [msa]
  deletion_matrices = [deletion_matrix]
  try:
  # gather features
    feature_dict = {
        **pipeline.make_sequence_features(sequence=query_sequence,
                                          description="none",
                                          num_res=len(query_sequence)),
        **pipeline.make_msa_features(msas=msas,deletion_matrices=deletion_matrices),
        **template_features
    }
  except:
    print(jobname+" cound not be processed")
    continue

  
  outs = predict_structure(jobname, feature_dict,
                           Ls=[len(query_sequence)], crop_len=crop_len,
                           model_params=model_params, use_model=use_model,
                           do_relax=use_amber)

  # gather MSA info
  deduped_full_msa = list(dict.fromkeys(msa))
  msa_arr = np.array([list(seq) for seq in deduped_full_msa])
  seqid = (np.array(list(query_sequence)) == msa_arr).mean(-1)
  seqid_sort = seqid.argsort() #[::-1]
  non_gaps = (msa_arr != "-").astype(float)
  non_gaps[non_gaps == 0] = np.nan

  # json out
  result={}
  for model_num in range(1,num_models+1):
    model_name = f"model_{model_num}"
    result[model_num]={}
    result[model_num]["plddt"]=outs[model_name]["plddt"].tolist()
    result[model_num]["pae"]=outs[model_name]["pae"]
    for index in range(len(outs[model_name]["pae"])):
      outs[model_name]["pae"][index]= outs[model_name]["pae"][index].tolist()
    result[model_num]["unrelaxedPDB"]=outs[model_name]["unrelaxedPdb"].split("\n");
    if "relaxedPdb" in outs[model_name]:
      result[model_num]["relaxedPDB"]=outs[model_name]["relaxedPdb"].split("\n");

  result["msa"]={}
  result["msa"]["coverage"]=(msa_arr != "-").sum(0).tolist()
  result["msa"]["alignment"]=deduped_full_msa

  result["id"]="_".join(jobname.split("_")[:-1])
  ends=jobname.split("_")[-1].split("-")
  result["start"]=int(ends[0])
  result["end"]=int(ends[1])
  result["seq"]=result["msa"]["alignment"][0]
  result["hash"]=hashlib.sha1(result["seq"].encode()).hexdigest()[:5]

  from datetime import datetime
  now=datetime.utcnow()
  epoch=datetime.utcfromtimestamp(0)
  result["metadata"]={};
  result["metadata"]["time"]=datetime.strftime(now, "%d/%m/%Y, %H:%M:%S")
  result["metadata"]["timeStamp"]=(now - epoch).total_seconds()
  result["metadata"]["predictedBy"]=user_name
  
  with open(f"{jobname}.json", "w") as jsonfile:
    json.dump(result, jsonfile, indent=2)

  print("Submitting the prediction to the community-generated database")
  %shell gzip -k $jobname".json"
  %shell curl -i -X POST "http://wheelerlab.net/submit.php" -s --data-binary "@"$jobname".json" -H "Content-Type: text/xml"
  print("http://wheelerlab.net/alphafold/all/view.php?idse="+jobname)

  plt.figure(figsize=(14,4),dpi=100)
  plt.subplot(1,2,1); plt.title("Sequence coverage")
  plt.imshow(non_gaps[seqid_sort]*seqid[seqid_sort,None],
             interpolation='nearest', aspect='auto',
             cmap="rainbow_r", vmin=0, vmax=1, origin='lower')
  plt.plot((msa_arr != "-").sum(0), color='black')
  plt.xlim(-0.5,msa_arr.shape[1]-0.5)
  plt.ylim(-0.5,msa_arr.shape[0]-0.5)
  plt.colorbar(label="Sequence identity to query",)
  plt.xlabel("Positions")
  plt.ylabel("Sequences")

  plt.subplot(1,2,2); plt.title("Predicted lDDT per position")
  for model_name,value in outs.items():
    plt.plot(value["plddt"],label=model_name)
  if homooligomer > 0:
    for n in range(homooligomer+1):
      x = n*(len(query_sequence)-1)
      plt.plot([x,x],[0,100],color="black")
  plt.legend()
  plt.ylim(0,100)
  plt.ylabel("Predicted lDDT")
  plt.xlabel("Positions")
  #plt.savefig(jobname+"_coverage_lDDT.png")

  plt.figure(figsize=(3*num_models,2), dpi=100)
  for n,(model_name,value) in enumerate(outs.items()):
    plt.subplot(1,num_models,n+1)
    plt.title(model_name)
    plt.imshow(value["pae"],label=model_name,cmap="bwr",vmin=0,vmax=30)
    plt.colorbar()
  #plt.savefig(jobname+"_PAE.png")

